In [301]:
import os

In [302]:
import pandas as pd

In [303]:
import folium

In [304]:
import geopandas as gpd

In [305]:
import webbrowser

In [306]:
import geojson

In [307]:
import json

In [308]:
from IPython.display import display

In [366]:
class Visualise:
    '''
    Class representing processed data from previous parts with attributes and methods to make geo-visualisations.
    '''
    def __init__(self, allowLog = True):
        '''
        Initilization of Downloader object. Storing objects from the webpage with self.qwe, self.rty and self.uio
        '''
        self.allowLog = allowLog
        if self.allowLog:
            print('Object of data initialized.')    
            
    def countriesPolygon(self, geoDf):
        '''
        Pandas GeoDataframe containing the (multi-)polygons of countries.
        '''
        self.geoDf = geoDf
    
    def foliumMap(self, m = folium.Map(location=[51.05, 5.86],zoom_start=3)):
        '''
        Map from library Folium.
        '''
        self.m = m         
    
    def countriesDict(self, dct):
        '''
        Dictionary for different naming conventions of countries in the World
        '''
        self.dct = {}
        for name, name2 in zip(dct['NAME'], dct['GEONAME']):
            self.dct[name] = name2
        #return self.dct
        # now the special care for a country with symbol ' in its name
        self.dct['Cote D\'Ivoire'] = 'Ivory Coast'
        self.data = self.data.replace(dct)
        
    def loadTheData(self, data, whatData = 'GeoTopArtist'):
        '''
        Loader of the data. First argument is the data provided, second is specification about what type of data it is.
        '''
        self.data = data
        self.whatData = whatData
        if whatData == 'GeoTopArtist':
            pass
        elif whatData == 'GeoTopTracks':
            pass
        elif whatData == 'ArtistInfo':
            pass
        else:
            return(print('I cannot recognize the type of data you want to pass. Select please from one of the following: \n\"GeoTopArtist\" \n\"GeoTopTracks\" \n\"ArtistInfo\" \nThank you very much!'))
        
    def showMeTopN(self, N = 10, whichOnes = 'Artists' ):
        '''
        Method which computes the top N artists for all countries contained in the scraped data.
        '''
        boundary = self.data['Rank'] <= N
        self.data = self.data.replace(self.dct)
        self.cols = ['Top ' + str(i+1) for i in range(N)]
        if whichOnes == 'Artists':
            for i in range(N):
                self.geoDf['Top '+ str(i+1)] = str(None)
            self.topNArtists = self.data[boundary]
            for cntry in self.topNArtists['Country']:
                for i in range(N):
                    try:
                        self.geoDf['Top ' + str(i+1)][self.geoDf['ADMIN']==cntry] = self.topNArtists[self.topNArtists['Country']==cntry][self.topNArtists['Rank']==i+1]['Artist'].item()
                    except:
                        pass
            return self.geoDf                               
        elif whichOnes == 'Tracks':
            for i in range(N):
                self.geoDf['Top '+ str(i+1)] = str(None)
            self.topNTracks = self.data[boundary]    
            for cntry in self.topNTracks['Country']:
                for i in range(N):
                    try:
                        self.geoDf['Top ' + str(i+1)][self.geoDf['ADMIN']==cntry] = self.topNTracks[self.topNTracks['Country']==cntry][self.topNTracks['Rank']==i+1]['Track'].item()
                    except:
                        pass
            return self.geoDf
        else:
            return(print('I cannot show you what you want..probably. Select please from one of the following: \n"Artists" \n"Tracks" \nThank you very much!'))
    
    def saveAndShowMap(self, m, df):
        '''
        Method to save and show the map of top artists or tracks. The map is saved as 'plotTheMap.html' file which is shown in a new window.
        '''
        folium.GeoJson(
                df[['ADMIN', 'geometry'] + self.cols].to_json(),
                show=True,
                tooltip=folium.features.GeoJsonTooltip(
                fields=['ADMIN'] + self.cols,
                aliases=['Country'] + self.cols)
            ).add_to(m)
        self.m.save('plotTheMap.html')
        webbrowser.open_new('plotTheMap.html')

pedro 4:54 PM

self.DfGeoTopArtists = pd.DataFrame(columns=['CountryId', 'Country', 'Rank', 'Artist', 'ArtistId', 'Listeners'])

self.DfGeoTopTracks = pd.DataFrame(columns=['CountryId', 'Country', 'Rank', 'Track', 'Duration', 'Artist', 'ArtistId'])
        
self.DfArtistInfo = pd.DataFrame(columns=['ArtistId', 'Artist', 'Listeners', 'Scrobbles', 'Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5'])

In [360]:
with open('countries.geojson', 'r') as f:
    data = geojson.load(f)

In [361]:
with open('countries_cut_two.csv', 'r') as c:
    cntrsDict = pd.read_csv(c, sep = ';')

In [367]:
newPD = Visualise()
newPD.foliumMap()
newPD.countriesPolygon(gpd.GeoDataFrame.from_features(data))
newPD.loadTheData(topArtists) # sem budu dávat data od Pedra
newPD.countriesDict(cntrsDict)
#newPD.dct

newPD.showMeTopN(N = 10, whichOnes = 'Artists')
newPD.saveAndShowMap(newPD.m,newPD.geoDf)
#newPD.data

Object of data initialized.


C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


---

### Debugging and testing

In [353]:
newPD.data[newPD.data['Country']=='Czech Republic']
newPD.data.head(47525)

,Country,Rank,Artist,Scrobbles
0,Taiwan,1,Coldplay,5486719
1,Taiwan,2,Adele,2939052
2,Taiwan,3,Ed Sheeran,1770632
3,Taiwan,4,Sia,2214287
4,Taiwan,5,Maroon 5,3609286
...,...,...,...,...
47520,Cote D'Ivoire,2,Michael Jackson,3638343
47521,Cote D'Ivoire,3,Bob Marley,2009135
47522,Cote D'Ivoire,4,Kanye West,4523577
47523,Cote D'Ivoire,5,Daft Punk,3864256


In [184]:
df = gpd.GeoDataFrame.from_features(data)

In [340]:
newPD.geoDf[newPD.geoDf['ADMIN']=="Ivory Coast"]

,geometry,ADMIN,ISO_A3,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
43,"MULTIPOLYGON (((-3.18497 5.13227, -3.16340 5.1...",Ivory Coast,CIV,None,None,None,None,None,None,None,None,None,None


In [343]:
newPD.geoDf[newPD.geoDf['ADMIN']=="United States of America"]

,geometry,ADMIN,ISO_A3,Top 1,Top 2,Top 3,Top 4,Top 5,Top 6,Top 7,Top 8,Top 9,Top 10
238,"MULTIPOLYGON (((-155.60652 20.13796, -155.5863...",United States of America,USA,None,None,None,None,None,None,None,None,None,None


,Unnamed: 0,ArtistId,Artist,Listeners,Scrobbles,Tag1,Tag2,Tag3,Tag4,Tag5
0,0,603ba565-3967-4be1-931e-9cb945394e86,*NSYNC,1000244,13392103,pop,boybands,90s,dance,seen live
1,1,f37c537b-3557-4031-bfd6-ab63ced32854,10cc,757032,6539113,classic rock,rock,70s,pop,british
2,2,dff0d392-4cd5-4052-9fbb-f485df3891e5,2 Chainz,689853,10755122,Hip-Hop,rap,Dirty South,trap,southern rap
3,3,02628cd8-ca14-4545-a0b4-96020b54692b,20syl,77619,892733,Hip-Hop,french,electronic,turntablism,hip hop
4,4,382f1005-e9ab-4684-afd4-0bdae4ee37f2,2Pac,2245429,58602220,rap,Hip-Hop,Gangsta Rap,West Coast Rap,hip hop


In [300]:
countriesLastfm = pd.read_csv('countries_cut_two.csv', sep = ';',encoding='latin-1')
countriesLastfm = countriesLastfm.drop(columns = ['COUNTRY'], axis = 1)
topArtists = pd.read_csv('CompleteArtists.csv', sep = ';')
topArtists = topArtists.drop(columns = ['Unnamed: 0'])

Geoname = [None] * (topArtists['Country'].count())
zmena = countriesLastfm['NAME'] != countriesLastfm['GEONAME']
countriesLastfm[zmena]
dct = {}
for name, name2 in zip(countriesLastfm['NAME'], countriesLastfm['GEONAME']):
    dct[name] = name2
 



In [189]:
topArtists = topArtists.replace(dct)

Editing some of the names in Geopandas dataframe due to different country naming conventions: create a dictionary.

In [211]:
cols = ['Top ' + str(i+1) for i in range(10)]
artists = ['Artist ' + str(i+1) for i in range(10)]
for i in range(len(artists)):
    df['Top '+ str(i+1)] = str(None)

In [212]:
for cntry in top10Artists['Country']:
    for i in range(10):
        try:
            df['Top ' + str(i+1)][df['ADMIN']==cntry] = top10Artists[top10Artists['Country']==cntry][top10Artists['Rank']==i+1]['Artist'].item()
        except:
            pass

C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: `item` has been deprecated and will be removed in a future version
  after removing the cwd from sys.path.
C:\Users\matej\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [208]:
folium.GeoJson(
    df[['ADMIN', 'geometry'] + cols].to_json(),
    show=True,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['ADMIN'] + cols,
        aliases=['Country'] + cols)
).add_to(m)

Unfortunately the map cannot be displayed in Jupyter notebook, but with help of _webbrowser_ library, we can save the output as html file and open it in a new window.

In [297]:
newPD.m.save('Top10OOPArtists.html')

In [298]:
webbrowser.open_new('Top10OOPArtists.html')

True

In [345]:
topArtists[topArtists['Country']=='Cote d\'Ivoire']

,Country,Rank,Artist,Scrobbles


In [369]:
import numpy as np

In [368]:
simulatedTracks =  pd.DataFrame(columns=['CountryId', 'Country', 'Rank', 'Track', 'Duration', 'Artist', 'ArtistId'])

for k in range(50):
    cId = k
    cntry = 'Country ' + str(k+1)
    for j in range(50):
        Rank